In [1]:
import os
import numpy as np
import nibabel as nib
from scans import *
from tumor_centers import *
from skimage.segmentation import morphological_chan_vese

In [2]:
def dice_coefficient(segmented, ground_truth):
    intersection = np.logical_and(segmented, ground_truth)
    return (2. * intersection.sum()) / (segmented.sum() + ground_truth.sum())

def select_slice(scan, label, slice_idx):
    return scan[:, :, slice_idx], label[:, :, slice_idx]

def load_data(scan_path, label_path):
    scan = nib.load(scan_path).get_fdata()
    label = nib.load(label_path).get_fdata()
    return scan, label

In [3]:
# File paths
dataset_path = 'D:\Dosyalar\dataset\Task06_Lung'
images_train_dir = os.path.join(dataset_path, 'imagesTr')
labels_train_dir = os.path.join(dataset_path, 'labelsTr')

In [4]:
# Results file
with open('results.txt', 'w') as results_file:
    for scan_number in INDEXES[-6:]:
        scan_path = os.path.join(images_train_dir, f'lung_{scan_number}.nii.gz')
        label_path = os.path.join(labels_train_dir, f'lung_{scan_number}.nii.gz')
        scan, label = load_data(scan_path, label_path)
        
        slice_indexes = globals()[f'scan_{scan_number}']
        tumor_centers = globals()[f'tumor_centers_{scan_number}']

        dscs = []
        sensitivities = []
        specificities = []
        accuracies = []
        weights = []

        for i, slice_idx in enumerate(slice_indexes):
            slice, slice_label = select_slice(scan, label, slice_idx)
            
            center = tumor_centers[i]
            seed_x, seed_y = int(center[0]), int(center[1])

            r = 10 
            initial_level_set = np.zeros_like(slice)
            X, Y = np.meshgrid(np.arange(slice.shape[1]), np.arange(slice.shape[0]))
            initial_level_set[((X - seed_x) ** 2 + (Y - seed_y) ** 2) < r ** 2] = 1

            segmented_mask = morphological_chan_vese(slice, 5, init_level_set=initial_level_set)
            # 5 was 100

            # Calculate TP, TN, FP, FN
            TP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 1))
            TN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 0))
            FP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 0))
            FN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 1))

            dsc = dice_coefficient(segmented_mask, slice_label)
            print(f'Scan {scan_number} - Slice {slice_idx}: {dsc}')
            sensitivity = TP / (TP + FN) if TP + FN > 0 else 0
            specificity = TN / (TN + FP) if TN + FP > 0 else 0
            accuracy = (TP + TN) / (TP + TN + FP + FN) if TP + TN + FP + FN > 0 else 0

            dscs.append(dsc)
            sensitivities.append(sensitivity)
            specificities.append(specificity)
            accuracies.append(accuracy)
            weights.append(np.sum(segmented_mask))

        weighted_mean_dsc = np.average(dscs, weights=weights)
        weighted_std_dsc = np.sqrt(np.average((dscs - weighted_mean_dsc) ** 2, weights=weights))
        mean_dsc = np.mean(dscs)  # Calculate mean DSC
        weighted_mean_sensitivity = np.average(sensitivities, weights=weights)
        weighted_std_sensitivity = np.sqrt(np.average((sensitivities - weighted_mean_sensitivity) ** 2, weights=weights))
        mean_sensitivity = np.mean(sensitivities)
        weighted_mean_specificity = np.average(specificities, weights=weights)
        weighted_std_specificity = np.sqrt(np.average((specificities - weighted_mean_specificity) ** 2, weights=weights))
        mean_specificity = np.mean(specificities)
        weighted_mean_accuracy = np.average(accuracies, weights=weights)
        weighted_std_accuracy = np.sqrt(np.average((accuracies - weighted_mean_accuracy) ** 2, weights=weights))
        mean_accuracy = np.mean(accuracies)
        
        results_file.write(f'Scan {scan_number}:\n'
                           f'  - Weighted Mean DSC: {weighted_mean_dsc}, '
                           f'Standard Deviation: {weighted_std_dsc}, Mean DSC: {mean_dsc}\n'
                           f'  - Weighted Mean Sensitivity: {weighted_mean_sensitivity}, '
                           f'Standard Deviation: {weighted_std_sensitivity}, Mean Sensitivity: {mean_sensitivity}\n'
                           f'  - Weighted Mean Specificity: {weighted_mean_specificity}, '
                           f'Standard Deviation: {weighted_std_specificity}, Mean Specificity: {mean_specificity}\n'
                           f'  - Weighted Mean Accuracy: {weighted_mean_accuracy}, '
                           f'Standard Deviation: {weighted_std_accuracy}, Mean Accuracy: {mean_accuracy}\n\n')

print("Results saved in 'results.txt'")

Scan 084 - Slice 161: 0.13839285714285715
Scan 084 - Slice 162: 0.12108559498956159
Scan 084 - Slice 163: 0.12678936605316973
Scan 084 - Slice 164: 0.22857142857142856
Scan 084 - Slice 165: 0.22657952069716775
Scan 084 - Slice 166: 0.15025906735751296
Scan 084 - Slice 167: 0.41595441595441596
Scan 084 - Slice 168: 0.44938271604938274
Scan 084 - Slice 169: 0.4779582366589327
Scan 084 - Slice 170: 0.5440613026819924
Scan 084 - Slice 171: 0.6928471248246845
Scan 084 - Slice 172: 0.6496945010183299
Scan 084 - Slice 173: 0.674264007597341
Scan 084 - Slice 174: 0.6698748796920115
Scan 084 - Slice 175: 0.6567685589519651
Scan 084 - Slice 176: 0.6560509554140127
Scan 084 - Slice 177: 0.6616334934797529
Scan 084 - Slice 178: 0.6921606118546845
Scan 084 - Slice 179: 0.6473684210526316
Scan 084 - Slice 180: 0.5410404624277456
Scan 084 - Slice 181: 0.45570866141732286
Scan 084 - Slice 182: 0.4286410131514856
Scan 084 - Slice 183: 0.4538087520259319
Scan 084 - Slice 184: 0.46835443037974683
Scan 08